# The Debugger

We use the Arduino Due board as a Hardware Debugger for the SWD debug protocol. SWD debug is an alternative by ARM to the JTAG debug protocol. The device under test can be any device with an SWD debug port. We use a second Arduino Due to demonstrate how it works.

## Wiring

Connect pin 3 and pin 4 of the Debugger with pins SWCLK and SWDIO of the DEBUG connector of the target.

![SWD Debug Wiring](pdf/SwdDebugWiring.jpg)

## Program the devices

Connect the Debugger and DUT to the USB board of your linux computer.
Assume that the Debugger is at `/dev/ttyACM0`, the DUT at `/dev/ttyACM1`. Run all shell commands from the `testbench` folder. To program the devices:

    make sam3x8e-flash PORT=/dev/ttyACM0
    make sam3x8e-flash PORT=/dev/ttyACM1

## Connect from the testbench

First change to the `testbench` folder:

In [4]:
cd ..

/home/frank/proj/testbench


In [5]:
import py

In [ ]:
l = py.loader.loader("/dev/ttyACM1")
s = py.Sam3xSerial("/dev/ttyACM0")
d = py.debug.debugger.Debugger(s)

In [6]:
d.is_halted()

NameError: name 'd' is not defined

In [ ]:
d.halt()
hex(d.R0())

# Use of source debugging functionality

In [1]:
import serial
import testbench
import testbench.debug

In [2]:
s = serial.Serial('/dev/ttyACM0', 115200, timeout=3)
t = testbench.Tester()
t.connect('/dev/ttyACM1')

In [3]:
d = testbench.debug.ELFDebugger('target/flash.elf', s, setup=True)

In [4]:
d.is_halted()

False

In [5]:
d.halt()

In [6]:
d.is_halted()

True

## Viewing source code

In [7]:
d.view_source()

/home/david/target/due/projects/example-apdu/lib/uart.c:23:0
     
     /** Receive one character. **/
     int uart_receive_uint8(Uart* p_uart, uint8_t *p) {
>>>  	while ((p_uart->UART_SR & 0x01) == 0) { }
     
     	*p = p_uart->UART_RHR;
     


In [8]:
d.resume()

In [9]:
d.is_halted()

False

## Setting Flash Patch Breakpoints (FPBs) and stepping through assembly code

In [10]:
bp1 = d.set_breakpoint_source('blake2s.S', 29)

Blake command will time out as the MCU will hit a breakpoint.

In [11]:
t.blake2s(0x20001000, 0x100)

Timeout: received 0 bytes in 3 seconds but expected 34

In [12]:
d.is_halted()

True

In [13]:
d.view_source()

/home/david/target/due/projects/example-apdu/target/blake2s.S:29:0
     @ h[0] ^= 0x0101000 ^ 32
     ldr R3, [R0, #0]
     mov R4, #0x01000000
>>>  add R4, #0x00010000
     add R4, #32
     eor R3, R4
     str R3, [R0, #0]


In [14]:
d.step()

In [15]:
d.view_source()

/home/david/target/due/projects/example-apdu/target/blake2s.S:30:0
     ldr R3, [R0, #0]
     mov R4, #0x01000000
     add R4, #0x00010000
>>>  add R4, #32
     eor R3, R4
     str R3, [R0, #0]
     @ free R3, R4


In [16]:
d.resume()

In [17]:
d.is_halted()

False

Clear the serial port buffer of the now-sent answer to the previously issued Blake command.

In [18]:
t.apdu.serial.read_all()

b'\xfb3?\xf4\x03\x08K\xb3=l\x19\x18+-\x0fH\xa6\xccW\xf2\x10\xf7Rd\xe9\xfexX\x8a\xf0\xe1\x0f\x00\x90'

## Setting source code breakpoints
Set a breakpoint at the start of the apdu echo command handler. (line 400 in `main.c`)

In [19]:
bp2 = d.set_breakpoint_source('main.c', 324)

In [20]:
d.is_halted()

False

In [21]:
t.echo(b'test')

Timeout: received 0 bytes in 3 seconds but expected 6

In [22]:
d.is_halted()

True

In [23]:
d.view_source()

/home/david/target/due/projects/example-apdu/target/main.c:324:0
     	}
     
     } else if (apdu.ins == 'E') {
>>>  	for (int i = 0; i < apdu.le; i++) {
     		uart_send_uint8(p_uart, apdu.data[i]);
     	}
     


In [24]:
d.env.apdu.ins

69

In [25]:
chr(69)

'E'

In [26]:
hex(d.env.p_uart)

'0x20087fa8'

In [27]:
d.resume()

In [28]:
d.is_halted()

False

In [29]:
d.unset_breakpoint(bp2)

In [30]:
t.apdu.serial.read_all()

b'test\x00\x90'

In [31]:
t.echo(b'testtwo')

b'testtwo'

# Links

[Arduino Due Pinout](https://docs.arduino.cc/resources/pinouts/A000056-full-pinout.pdf?_gl=1*87z8vp*_up*MQ..*_ga*ODk2NjY4NTY2LjE3Mzg4MzAwODQ.*_ga_NEXN8H46L5*MTczODgzMDA4MS4xLjAuMTczODgzMDA4MS4wLjAuMTMwMzUyMTYxMA..)
